In [16]:
try:
    import cv2
    import numpy as np
    import mediapipe
    import matplotlib.pyplot as plt
    import pyautogui
    import mouse
    import traceback
    import math
    capture = cv2.VideoCapture(0)
    fw = capture.get(cv2.CAP_PROP_FRAME_WIDTH)
    fh = capture.get(cv2.CAP_PROP_FRAME_HEIGHT)

    def show(i):
        if i is not None:
            coor = mediapipe.solutions.drawing_utils._normalized_to_pixel_coordinates(i.x,i.y,fw,fh)
            if coor is not None:
                cv2.putText(frame,str(coor),(coor[0]+20,coor[1]+20),cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0, 0), 2)

    with mediapipe.solutions.hands.Hands(static_image_mode=False, min_detection_confidence=0.7, min_tracking_confidence=0.7, max_num_hands=1) as hands:
        while capture.isOpened():
            ret, frame = capture.read()
            if ret == False:
                continue
            frame = cv2.flip(frame, 1)
            results = hands.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
            if results.multi_hand_landmarks != None:
                thumb=results.multi_hand_landmarks[0].landmark[mediapipe.solutions.hands.HandLandmark.THUMB_TIP]
                index = results.multi_hand_landmarks[0].landmark[mediapipe.solutions.hands.HandLandmark.INDEX_FINGER_TIP]
                middle = results.multi_hand_landmarks[0].landmark[mediapipe.solutions.hands.HandLandmark.MIDDLE_FINGER_TIP]
                ring= results.multi_hand_landmarks[0].landmark[mediapipe.solutions.hands.HandLandmark.RING_FINGER_TIP]
                little=results.multi_hand_landmarks[0].landmark[mediapipe.solutions.hands.HandLandmark.PINKY_TIP]
                a=[thumb,index,middle,ring,little]
                x0=0
                y0=0
                for i in a:
                    x0+=i.x
                    y0+=i.y
                c=(x0/5,y0/5)
                centroid = mediapipe.solutions.drawing_utils._normalized_to_pixel_coordinates(c[0],c[1],fw,fh)
                frame=cv2.circle(frame, centroid, 10, (0, 0, 255), -1)
                dis=[math.dist(mediapipe.solutions.drawing_utils._normalized_to_pixel_coordinates(i.x,i.y,fw,fh),centroid) for i in a]
                disavg=sum(dis)/5
                print(dis,disavg)
                show(thumb)
                show(index)
                show(middle)
                show(ring)
                show(little)
            cv2.imshow('Mouse Controller', frame)
            if cv2.waitKey(1) == 27: #escape key
                break
    cv2.destroyAllWindows()
    capture.release()

except:
    cv2.destroyAllWindows()
    capture.release()
    traceback.print_exc()

[74.84650960465692, 33.12099032335839, 47.885279575251516, 29.154759474226502, 45.34313619501853] 46.07013503450237
[71.16881339463234, 38.62641583165593, 53.33854141237835, 31.112698372208094, 45.34313619501853] 47.917921041178644
[82.92767933567175, 30.610455730027933, 27.85677655436824, 39.824615503479755, 50.921508225896055] 46.42820706988875
[85.80209787645055, 33.24154027718932, 30.675723300355937, 38.01315561749642, 52.49761899362675] 48.0460272130238
[85.21150157109074, 33.24154027718932, 30.675723300355937, 38.47076812334269, 52.49761899362675] 48.01943045312109
[85.0, 32.55764119219941, 29.732137494637012, 38.47076812334269, 51.54609587543949] 47.46132853712372
[85.42833253669417, 33.421549934136806, 30.083217912982644, 37.589892258425, 53.160135440008055] 47.936625616449334
[84.81155581640984, 33.421549934136806, 30.675723300355937, 38.94868418830089, 51.54609587543949] 47.88072182292859
[84.81155581640984, 33.60059523282288, 30.675723300355937, 38.07886552931954, 51.5460958